In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn.metrics import accuracy_score

## Load data

In [2]:
dir_data = "/Users/romulo/Documents/Dataset/Earthquake Damage/"
#dir_data = "Dataset/"
df_x_train = pd.read_csv(dir_data+"train_values.csv",index_col="building_id")
df_y_train = pd.read_csv(dir_data+"train_labels.csv",index_col="building_id")
df_x_test = pd.read_csv(dir_data+"test_values.csv",index_col="building_id")

#df_x_train = df_x_train[:100]
#df_y_train = df_y_train[:100]
#df_x_test = df_x_test[:100]

data_train = df_x_train.merge(df_y_train, how='left', left_index=True, right_index=True)



print("len train:",len(data_train))
print("len train:",len(df_x_test))

len train: 260601
len train: 86868


## Preparate data

In [3]:
# get y train
y_train = data_train['damage_grade'].values

# remove y of data_train
data_train = data_train.drop('damage_grade', 1)

In [4]:
# let's put the train data and test data together to make get_dummies and then divide
df_x_all = data_train.append(df_x_test)
print("len all:",len(df_x_all))

# get dummies from cat columns
cat_var = [key for key in dict(df_x_all.dtypes) if dict(df_x_all.dtypes)[key] in ['object'] ]
df_x_all = pd.get_dummies(df_x_all, prefix=cat_var, columns=cat_var)

#divide x_train and x_test
x_train = df_x_all.iloc[:len(data_train)]
x_test = df_x_all.iloc[len(data_train):]

len all: 347469


In [5]:
# clear memory
del df_x_all,df_x_train,df_y_train,data_train,df_x_test

In [6]:
#get x_dev and y_dev (10% from train)
x_train, x_dev, y_train, y_dev = train_test_split( x_train, y_train, test_size=0.1, random_state=42)

In [7]:
print("len x_train : %d  len y_train: %d " %(len(x_train),len(y_train)) )
print("len x_dev   : %d  len y_dev  : %d " %(len(x_dev),len(y_dev)) )
print("len x_test  : %d" %(len(x_test)) )

len x_train : 234540  len y_train: 234540 
len x_dev   : 26061  len y_dev  : 26061 
len x_test  : 86868


## Testing Algorithms from sklearn

In [8]:
names = ["Nearest Neighbors", "Decision Tree", "Random Forest", "Neural Net", "AdaBoost","Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(10),
    DecisionTreeClassifier(max_depth=8),
    RandomForestClassifier(max_depth=8, n_estimators=1500, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [9]:
df_results_sklearn = pd.DataFrame(columns=["algorithm","acc_train","acc_dev","precision_1","recall_1","f1-score_1","precision_2","recall_2","f1-score_2","precision_3","recall_3","f1-score_3"])
for name, clf in zip(names, classifiers):
        print(name)
        clf.fit(x_train, y_train)
        score_train = clf.score(x_train, y_train)
        score_dev = clf.score(x_dev, y_dev)
        print("\tscore_train: ",score_train)
        print("\tscore_dev: ",score_dev)
        print("\n\n")
        
        pred_y_pred = clf.predict(x_dev)
        
        dict_report = classification_report(y_dev, pred_y_pred,output_dict=True)
        precision_1 = dict_report["1"]["precision"]
        precision_2 = dict_report["2"]["precision"]
        precision_3 = dict_report["3"]["precision"]
        
        recall_1 = dict_report["1"]["recall"]
        recall_2 = dict_report["2"]["recall"]
        recall_3 = dict_report["3"]["recall"]
        
        score_1 = dict_report["1"]["f1-score"]
        score_2 = dict_report["2"]["f1-score"]
        score_3 = dict_report["3"]["f1-score"]
        
        df_results_sklearn = df_results_sklearn.append({"algorithm": name,"acc_train":score_train,"acc_dev":score_dev,"precision_1" : precision_1, "recall_1" : recall_1, "f1-score_1" : score_1,"precision_2" : precision_2, "recall_2" : recall_2, "f1-score_2" : score_2,"precision_3" : precision_3, "recall_3" : recall_3, "f1-score_3" : score_3} , ignore_index=True)
        df_results_sklearn.to_csv("results/df_results_sklearn_alg.csv")
        

Nearest Neighbors
	score_train:  0.7590943975441289
	score_dev:  0.7123671386362764



Decision Tree
	score_train:  0.6749765498422444
	score_dev:  0.6734584244656767



Random Forest
	score_train:  0.569143856058668
	score_dev:  0.5676297916426845





/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Neural Net
	score_train:  0.5732241835081436
	score_dev:  0.573539004642953



AdaBoost
	score_train:  0.6452801227935533
	score_dev:  0.6440658455162888



Naive Bayes
	score_train:  0.43056621471817175
	score_dev:  0.43022140362994515



QDA


/usr/local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


	score_train:  0.37562036326426196
	score_dev:  0.37515828249107863





## Using XGBoost

In [ ]:
# My gridsearch (greedy)

df_results_XGBoost = pd.DataFrame(columns=["acc_train","acc_dev","max_depth","n_estimator","precision_1","recall_1","f1-score_1","precision_2","recall_2","f1-score_2","precision_3","recall_3","f1-score_3"])

max_depth = 6
n_estimators =  range(100,2100,100)

for n_estimator in n_estimators:
    print("n_estimators:",n_estimator)

    model = xgb.XGBClassifier(n_estimators=n_estimator,max_depth=max_depth)
    model.fit(x_train, y_train)

    y_train_pred = model.predict(x_train)
    y_dev_pred = model.predict(x_dev)
    

    accuracy_train = accuracy_score(y_train, y_train_pred)
    accuracy_dev = accuracy_score(y_dev, y_dev_pred)
    
    pred_y_pred = clf.predict(x_dev)
        
    dict_report = classification_report(y_dev, pred_y_pred,output_dict=True)
    precision_1 = dict_report["1"]["precision"]
    precision_2 = dict_report["2"]["precision"]
    precision_3 = dict_report["3"]["precision"]

    recall_1 = dict_report["1"]["recall"]
    recall_2 = dict_report["2"]["recall"]
    recall_3 = dict_report["3"]["recall"]

    score_1 = dict_report["1"]["f1-score"]
    score_2 = dict_report["2"]["f1-score"]
    score_3 = dict_report["3"]["f1-score"]
    
    df_results_XGBoost = df_results_XGBoost.append({"acc_train":accuracy_train,"acc_dev":accuracy_dev,"max_depth":max_depth, "n_estimator":n_estimator, "precision_1" : precision_1, "recall_1" : recall_1, "f1-score_1" : score_1,"precision_2" : precision_2, "recall_2" : recall_2, "f1-score_2" : score_2,"precision_3" : precision_3, "recall_3" : recall_3, "f1-score_3" : score_3} , ignore_index=True)
    df_results_XGBoost.to_csv("results/df_results_XGBoost.csv")
    del model
    print("\taccuracy_train",accuracy_train)
    print("\taccuracy_dev",accuracy_dev)
        

n_estimators: 100
